In [1]:
# === Library Import ===
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path

In [3]:
# === Configuration ===
ROOT = Path.cwd()
DATA_PATH = ROOT / "data" / "customer_data.csv"
OUTPUT_DIR = ROOT / "output"
OUTPUT_DIR.mkdir(exist_ok=True)

#以下、公開用のためコメントアウト
#print("参照データ：", DATA_PATH)
print("参照データ：customer_data.csv")

参照データ：customer_data.csv


In [5]:
# === Load Data ===
df = pd.read_csv(DATA_PATH, parse_dates=["Date"])
print("✅ データ読み込み完了")
print(df.head())

# === 基準日（最新日） ===
ref_date = df["Date"].max() + pd.Timedelta(days=1)

✅ データ読み込み完了
  CustomerID       Date   Amount
0      C0001 2025-01-06   7400.0
1      C0001 2025-01-18  29500.0
2      C0001 2025-01-25  28100.0
3      C0001 2025-01-31  14700.0
4      C0001 2025-02-19  13100.0


In [6]:
# === RFM集計 ===
rfm = (
    df.groupby("CustomerID")
    .agg({
        "Date": lambda x: (ref_date - x.max()).days,  # R
        "CustomerID": "count",                        # F
        "Amount": "mean",                             # M
    })
    .rename(columns={"Date": "Recency", "CustomerID": "Frequency", "Amount": "Monetary"})
    .reset_index()
)

In [7]:
# === スコア化（1〜5段階） ===
rfm["R_Score"] = pd.qcut(rfm["Recency"], 5, labels=[5,4,3,2,1]).astype(int)
rfm["F_Score"] = pd.qcut(rfm["Frequency"].rank(method="first"), 5, labels=[1,2,3,4,5]).astype(int)
rfm["M_Score"] = pd.qcut(rfm["Monetary"], 5, labels=[1,2,3,4,5]).astype(int)


In [11]:
# === 合計スコアと分類 ===
rfm["Total_Score"] = rfm[["R_Score", "F_Score", "M_Score"]].sum(axis=1)

# セグメント分類ルール
def segment(score):
    if score >= 13:
        return "優良顧客"
    elif score >= 10:
        return "リピーター"
    elif score >= 7:
        return "一般顧客"
    elif score >= 4:
        return "離脱リスク"
    else:
        return "休眠顧客"

rfm["Segment"] = rfm["Total_Score"].apply(segment)

# === Export ===
out_path = OUTPUT_DIR / "rfm_result.xlsx"
rfm.to_excel(out_path, index=False)

print("\n✅ RFM分析完了")

#print("📦 出力ファイル：", out_path)

print("📦 出力ファイル:output")
print("\n📈 サンプル出力：")
print(rfm.head(10))


✅ RFM分析完了
📦 出力ファイル:output

📈 サンプル出力：
  CustomerID  Recency  Frequency      Monetary  R_Score  F_Score  M_Score  \
0      C0001        9         26  14169.230769        3        3        2   
1      C0002       18         32  13846.875000        2        5        1   
2      C0003       29         17  18352.941176        1        1        5   
3      C0004       20         29  16700.000000        1        4        4   
4      C0005        9         30  15530.000000        3        5        3   
5      C0006       76         20  14565.000000        1        1        2   
6      C0007        4         28  18692.857143        4        4        5   
7      C0008       20         26  15515.384615        1        3        3   
8      C0009        1         28  11803.571429        5        4        1   
9      C0010       19         25  14768.000000        2        3        2   

   Total_Score Segment  
0            8    一般顧客  
1            8    一般顧客  
2            7    一般顧客  
3            9